In [ ]:
# Creates Dataset where values are for the probabilistic contingency table

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import cartopy as cp
from matplotlib.patches import FancyArrowPatch
from scipy.ndimage import map_coordinates
import metpy.calc as mpcalc
from geographiclib.geodesic import Geodesic
from utils_datetime import *
from utils_filter import *

In [2]:
# load grid pph and outlook
pph = xr.open_dataset('data/pph/labelled2023_pph.nc')
outlooks = xr.open_dataset('data/outlooks/grid_outlooks.nc')


In [ ]:
pph

In [3]:
# dataset in time, a/b/c/d, hazard
hazard_types= ['Wind', 'Hail', 'Tornado', 'All Hazard']
contingency_dataset = xr.Dataset(
    data_vars=dict(
        a=(['time', 'hazard'], np.full((len(pph['time']), len(hazard_types)), 0.0)),
        b=(['time', 'hazard'], np.full((len(pph['time']), len(hazard_types)), 0.0)),
        c=(['time', 'hazard'], np.full((len(pph['time']), len(hazard_types)), 0.0)),
        d=(['time', 'hazard'], np.full((len(pph['time']), len(hazard_types)), 0.0))
    ),
    coords=dict(
        time=(['time'], pph['time'].data),
        hazard=(['hazard'], hazard_types)
    ),
    attrs=dict(description="Number of a, b, c, and d in contingency tables for each day and hazard type, as calculated probabilistically from outlook and PPH probabilities")
)

In [4]:
pph_key_dict = {
    'Wind': 'p_perfect_wind',
    'Hail': 'p_perfect_hail',
    'Tornado': 'p_perfect_tor',
    'All Hazard': 'p_perfect_max'
}

outlook_key_dict = {
    'Wind': 'Day 1 Wind',
    'Hail': 'Day 1 Hail',
    'Tornado': 'Day 1 Tornado',
    'All Hazard': 'Day 1'
}

oldyear = 0

In [ ]:
for time in pph['time']:
    year = str(time.values)[:4]
    if year != oldyear:
        oldyear = year
        print(year)
    for hazard in hazard_types:
        o = outlooks.sel(time = time, outlook = outlook_key_dict[hazard])['prob'].values
        p = pph.sel(time = time)[pph_key_dict[hazard]].values/100

        a = np.multiply(o, p).sum()
        b = np.multiply(o, (1-p)).sum()
        c = np.multiply((1-o), p).sum()
        d = np.multiply((1-o), (1-p)).sum()

        contingency_dataset['a'].loc[dict(time = time, hazard = hazard)] = a
        contingency_dataset['b'].loc[dict(time = time, hazard = hazard)] = b
        contingency_dataset['c'].loc[dict(time = time, hazard = hazard)] = c
        contingency_dataset['d'].loc[dict(time = time, hazard = hazard)] = d

        
        



In [ ]:
contingency_dataset.to_netcdf('data/contingency/contingency.nc')